# Title: 

### Author: Cera Drake
### Date: 03/30/2025
#### Objective: 

##### Section 1: Import and Inspect the Data

In [1]:
import seaborn as sns
import pandas as pd

In [2]:
# Load Titanic dataset
titanic = sns.load_dataset('titanic')

#### Section 2: Data Exploration and Preparation

#### 2.1 Handle Missing Values and Clean Data
#### Impute missing values for age using the median:

In [3]:
titanic['age'].fillna(titanic['age'].median(), inplace=True)

C:\Users\cerad\AppData\Local\Temp\ipykernel_2356\3290119062.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['age'].fillna(titanic['age'].median(), inplace=True)


#### 2.1 Fill in missing values for embark_town using the mode:

In [4]:
titanic['embark_town'].fillna(titanic['embark_town'].mode()[0], inplace=True)

C:\Users\cerad\AppData\Local\Temp\ipykernel_2356\3403561138.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['embark_town'].fillna(titanic['embark_town'].mode()[0], inplace=True)


#### 2.2 Feature Engineering
##### Create new features:

###### Add family_size - number of family members on board
###### Convert categorical "sex" to numeric
###### Convert categorical "embarked" to numeric
###### Binary feature - convert "alone" to numeric

In [5]:
# Create new features
titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1
titanic['sex'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic['embarked'] = titanic['embarked'].map({'C': 0, 'Q': 1, 'S': 2})
titanic['alone'] = titanic['alone'].astype(int)

#### Section 3. Feature Selection and Justification

#### 3.1 Choose features and target
Select two or more input features (numerical for regression, numerical and/or categorical for classification)
Use survived as the target. 
We will do three input cases like the example. 
First:

input features: alone
target: survived
Second:

input features - age (or another variable of your choice)
target: survived
Third:

input features -  age and family_size (or another combination of your choice)
target: survived
 

#### 3.2 Define X (features) and y (target)
-Assign input features to X a pandas DataFrame with 1 or more input features

-Assign target variable to y (as applicable) - a pandas Series with a single target feature

-Again - use comments to run a single case at a time

-The follow starts with only the statements needed for case 1. 

Double brackets [[ ]]]  makes a 2D DataFrame
Single brackets [ ]  make a 1D Series

In [6]:
# Case 1: alone only 
X = titanic[['alone']]
y = titanic['survived']

# Case 2: age only (or your choice)
X = titanic[['age']]
y = titanic['survived']

# Case 3: age + family_size (or your choice)
X = titanic[['age', 'family_size']]
y = titanic['survived']